Auteur: Michaël Leroy

    Création  geodataframe des Noeuds du graph:
        attributs:
          * osmid : id openstreetmap de ref de la commune  <- index
          * codegeo: code insee pour pouvoir requeter sur les autres datasets
          * y: longitude point central de la commune
          * x: lattitude point centra de la commune	

          * counts: décompte du nombre de tags label 1 contenus dans le polygone de la commune
          * geometry: area shape de la commune

        ( A reprendre ou a requeter dans les datasets consolidés ?)
            pop:  nombre d'habitants
            bornes: 
            puissances:
            ....

  Les edges sont crées entre le centre commune et les 5 plus proches centres voisins (BallTree sklearn, distance Haversine), mais sans attributs à ce stade


  [TODO] vérifier si il ne faut pas mettre (v,u,1) quand (u,v,0)
          existe pour tenir compte de l'eventuelle non réciprocité des trajets routier
          en fonction des voies empruntées. En théorie pas nécessaire dataframe avec multi index,
          mais....


In [31]:
## For env version problem check

# import os
# os.environ['USE_PYGEOS'] = '0'
# import geopandas as gpd
# import osmnx as ox
# import networkx as nx
# import pyproj
# from shapely.ops import transform


In [32]:
class CFG:
    area = [
        'Auvergne-Rhône-Alpes',
    'Bourgogne-Franche-Comté',
    'Brittany',
    'Centre-Val de Loire',
    'Grand Est',
    'Hauts-de-France',
    'Île-de-France',
    'Normandie',
    'Nouvelle-Aquitaine',
    'Occitanie',
    'Pays de la Loire',
    'Provence-Alpes-Côte d\'Azur'
]
    area_name = 'france'      #'_'.join(area)

    

    bornes = False

    viz = True

    roads = (
        '|motorway'
        '|motorway_link'
        '|trunk'
        '|trunk_link'
        '|primary'
        '|primary_link'
        '|secondary'
        '|secondary_link'
        '|tertiary'
         '|tertiary_link'
    )

        # '|primary'
        # '|motorway_link'
        
        # '|primary_link'

        # '|secondary'
        # '|secondary_link'
        
        
        #  '|tertiary'
        #  '|tertiary_link'

    data_path = './osm_datas/'

import joblib
joblib.dump(CFG.area, 'CFG.pkl')    

import os
os.environ['USE_PYGEOS'] = '0'

%load_ext autoreload
%autoreload


%aimport OSM_helper_functions
%aimport OCM_helper_functions

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import osmnx as ox
import networkx as nx
from tqdm.notebook import tqdm
from OSM_helper_functions import *
from OCM_helper_functions import *


ox.settings.log_console=True
ox.settings.use_cache=True
''' To retrieve http data from OSMnx and cache them locally
    before building graph. Raise and exception if requests
    are all done. intercept then set to False and retry.'''
# ox.settings.cache_only_mode=True 

Administratives = {
    'Auvergne-Rhône-Alpes': ['Ain', 'Allier', 'Ardèche', 'Cantal', 'Drôme', 'Isère', 'Loire', 'Haute-Loire', 'Puy-de-Dôme', 'Rhône', 'Savoie', 'Haute-Savoie'],
    'Bourgogne-Franche-Comté': ['Côte-d\'Or', 'Doubs', 'Jura', 'Nièvre', 'Haute-Saône', 'Saône-et-Loire', 'Yonne', 'Territoire de Belfort'],
    'Brittany': ['Côtes-d\'Armor', 'Finistère', 'Ille-et-Vilaine', 'Morbihan'],
    'Centre-Val de Loire': ['Cher', 'Eure-et-Loir', 'Indre', 'Indre-et-Loire', 'Loir-et-Cher', 'Loiret'],
    # 'Corse': ['Corse-du-Sud', 'Haute-Corse'],
    'Grand Est': ['Ardennes', 'Aube', 'Marne', 'Haute-Marne', 'Meurthe-et-Moselle', 'Meuse', 'Moselle', 'Bas-Rhin', 'Haut-Rhin', 'Vosges'],
    'Hauts-de-France': ['Aisne', 'Nord', 'Oise', 'Pas-de-Calais', 'Somme'],
    'Île-de-France': ['Paris', 'Seine-et-Marne', 'Yvelines', 'Essonne', 'Hauts-de-Seine', 'Seine-Saint-Denis', 'Val-de-Marne', 'Val-d\'Oise'],
    'Normandie': ['Calvados', 'Eure', 'Manche', 'Orne', 'Seine-Maritime'],
    'Nouvelle-Aquitaine': ['Charente', 'Charente-Maritime', 'Corrèze', 'Creuse', 'Dordogne', 'Gironde', 'Landes', 'Lot-et-Garonne', 'Pyrénées-Atlantiques', 'Deux-Sèvres', 'Vienne', 'Haute-Vienne'],
    'Occitanie': ['Ariège', 'Aude', 'Aveyron', 'Gard', 'Haute-Garonne', 'Gers', 'Hérault', 'Lot', 'Lozère', 'Hautes-Pyrénées', 'Pyrénées-Orientales', 'Tarn', 'Tarn-et-Garonne'],
    'Pays de la Loire': ['Loire-Atlantique', 'Maine-et-Loire', 'Mayenne', 'Sarthe', 'Vendée'],
    'Provence-Alpes-Côte d\'Azur': ['Alpes-de-Haute-Provence', 'Hautes-Alpes', 'Alpes-Maritimes', 'Bouches-du-Rhône', 'Var', 'Vaucluse']
}

Regions = [
    'Auvergne-Rhône-Alpes',
    'Bourgogne-Franche-Comté',
    'Brittany',
    'Centre-Val de Loire',
    # 'Corse',
    'Grand Est',
    'Hauts-de-France',
    'Île-de-France',
    'Normandie',
    'Nouvelle-Aquitaine',
    'Occitanie',
    'Pays de la Loire',
    'Provence-Alpes-Côte d\'Azur'
]

Name2num = {
    'Ain': '01',
    'Aisne': '02',
    'Allier': '03',
    'Alpes-de-Haute-Provence': '04',
    'Hautes-Alpes': '05',
    'Alpes-Maritimes': '06',
    'Ardèche': '07',
    'Ardennes': '08',
    'Ariège': '09',
    'Aube': '10',
    'Aude': '11',
    'Aveyron': '12',
    'Bouches-du-Rhône': '13',
    'Calvados': '14',
    'Cantal': '15',
    'Charente': '16',
    'Charente-Maritime': '17',
    'Cher': '18',
    'Corrèze': '19',
    # 'Corse-du-Sud': '2A',
    # 'Haute-Corse': '2B',
    'Côte-d\'Or': '21',
    'Côtes-d\'Armor': '22',
    'Creuse': '23',
    'Dordogne': '24',
    'Doubs': '25',
    'Drôme': '26',
    'Eure': '27',
    'Eure-et-Loir': '28',
    'Finistère': '29',
    'Gard': '30',
    'Haute-Garonne': '31',
    'Gers': '32',
    'Gironde': '33',
    'Hérault': '34',
    'Ille-et-Vilaine': '35',
    'Indre': '36',
    'Indre-et-Loire': '37',
    'Isère': '38',
    'Jura': '39',
    'Landes': '40',
    'Loir-et-Cher': '41',
    'Loire': '42',
    'Haute-Loire': '43',
    'Loire-Atlantique': '44',
    'Loiret': '45',
    'Lot': '46',
    'Lot-et-Garonne': '47',
    'Lozère': '48',
    'Maine-et-Loire': '49',
    'Manche': '50',
    'Marne': '51',
    'Haute-Marne': '52',
    'Mayenne': '53',
    'Meurthe-et-Moselle': '54',
    'Meuse': '55',
    'Morbihan': '56',
    'Moselle': '57',
    'Nièvre': '58',
    'Nord': '59',
    'Oise': '60',
    'Orne': '61',
    'Pas-de-Calais': '62',
    'Puy-de-Dôme': '63',
    'Pyrénées-Atlantiques': '64',
    'Hautes-Pyrénées': '65',
    'Pyrénées-Orientales': '66',
    'Bas-Rhin': '67',
    'Haut-Rhin': '68',
    'Rhône': '69',
    'Haute-Saône': '70',
    'Saône-et-Loire': '71',
    'Sarthe': '72',
    'Savoie': '73',
    'Haute-Savoie': '74',
    'Paris': '75',
    'Seine-Maritime': '76',
    'Seine-et-Marne': '77',
    'Yvelines': '78',
    'Deux-Sèvres': '79',
    'Somme': '80',
    'Tarn': '81',
    'Tarn-et-Garonne': '82',
    'Var': '83',
    'Vaucluse': '84',
    'Vendée': '85',
    'Vienne': '86',
    'Haute-Vienne': '87',
    'Vosges': '88',
    'Yonne': '89',
    'Territoire de Belfort': '90',
    'Essonne': '91',
    'Hauts-de-Seine': '92',
    'Seine-Saint-Denis': '93',
    'Val-de-Marne': '94',
    'Val-d\'Oise': '95'
}


AREA = []
for area in CFG.area:
    AREA.extend(Administratives[area] )
# AREA

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
data_path = './datas'
file_name = 'decoupage_administratif'


communes = gpd.read_feather(os.path.join(data_path,f'[{CFG.area_name}]{file_name}_osmid_centre.feather'))

communes.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [34]:
metro = Name2num.values()
communes = communes.query("dep in @metro")

Dirty fix on errors in osmid from previous notebook

# TODO : enquire why, move fix to prevoius notebook

In [35]:
# MANUAL FIX FOR MISSING OSMID ON Nominatim resquests failures
communes.loc[5228 , ['osmid','x','y']] = (89493,45.34241,0.14665)
communes.loc[5229 , ['osmid','x','y']] = (63211,45.32810,-0.19941)
communes.loc[10916, ['osmid','x','y']] = (59355,44.000480,4.368443)
communes.loc[34954, ['osmid','x','y']] = (78755,48.21385,-0.21375)

In [36]:
dup = communes[communes[['osmid']].duplicated()]
dup

,codgeo,dep,reg,xcl2154,ycl2154,libgeo,geometry,osmid,x,y
1039,02675,02,32,756530,6931643,Sainte-Croix,"POLYGON ((3.77080 49.48567, 3.76486 49.48064, ...",2953115.0,49.4809517,3.7781066
1073,02712,02,32,741504,6898258,Sergy,"POLYGON ((3.60190 49.18196, 3.60124 49.18692, ...",2955576.0,49.1828833,3.5710256
5553,17007,17,75,400068,6571413,Anais,"POLYGON ((-0.90061 46.18719, -0.90721 46.19125...",117882.0,46.1870947,-0.9078872
5588,17044,17,75,419598,6511251,Berneuil,"POLYGON ((-0.62773 45.62821, -0.62584 45.62795...",138512.0,45.6429064,-0.600994
5663,17122,17,75,433264,6506895,Coulonges,"POLYGON ((-0.43203 45.62467, -0.43911 45.62164...",152685.0,45.6092373,-0.4237379
5693,17154,17,75,408066,6528371,Les Essards,"POLYGON ((-0.75586 45.77929, -0.74841 45.77929...",125625.0,45.7917501,-0.7587287
5751,17215,17,75,428341,6491622,Lussac,"POLYGON ((-0.47459 45.45942, -0.47100 45.45878...",154481.0,45.469752,-0.476925
5763,17227,17,75,420035,6501542,Mazerolles,"POLYGON ((-0.61499 45.57141, -0.61348 45.56877...",152689.0,45.5481449,-0.581715
5765,17229,17,75,441679,6474613,Mérignac,"POLYGON ((-0.28163 45.33836, -0.29512 45.34106...",124631.0,45.3241667,-0.2943867
5774,17239,17,75,440267,6530043,Mons,"POLYGON ((-0.29918 45.80209, -0.30135 45.80614...",154522.0,45.8213116,-0.3471728


In [37]:
for n, idx in enumerate(dup.index):
    communes.loc[idx,'osmid'] = n + 10

In [38]:
dup = communes[communes[['osmid']].duplicated()]
dup

,codgeo,dep,reg,xcl2154,ycl2154,libgeo,geometry,osmid,x,y


In [39]:
# # Test on a sample
# deps = ['56']  #+['51','52','08','10']
# communes = communes.query("dep in @deps")



In [40]:


display(communes.head(1))
display(communes.info())
communes.shape

,codgeo,dep,reg,xcl2154,ycl2154,libgeo,geometry,osmid,x,y
0,01001,01,84,848241,6563021,L'Abergement-Clémenciat,"POLYGON ((4.90497 46.16079, 4.90280 46.15795, ...",142240.0,46.1502427,4.9202187


<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 34466 entries, 0 to 34954
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   codgeo    34466 non-null  object  
 1   dep       34466 non-null  object  
 2   reg       34466 non-null  object  
 3   xcl2154   34466 non-null  int64   
 4   ycl2154   34466 non-null  int64   
 5   libgeo    34466 non-null  object  
 6   geometry  34466 non-null  geometry
 7   osmid     34466 non-null  float64 
 8   x         34466 non-null  object  
 9   y         34466 non-null  object  
dtypes: float64(1), geometry(1), int64(2), object(6)
memory usage: 3.9+ MB


None

(34466, 10)

In [41]:
communes[communes.x.isna()]

,codgeo,dep,reg,xcl2154,ycl2154,libgeo,geometry,osmid,x,y


In [42]:
communes['osmid'] = communes['osmid'].astype(int)
communes['osmid'] = communes['osmid'].astype(str)

C:\Users\demo\AppData\Roaming\Python\Python39\site-packages\pandas\core\dtypes\astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)


In [43]:
# candidate_labels = joblib.load( os.path.join(CFG.data_path,f'[{CFG.area_name}]candidate_labels.pkl'))
# labels2legend = joblib.load( os.path.join(CFG.data_path,f'[{CFG.area_name}]labels2legend.pkl'))

# file_name = 'datatourisme-place-20230529-tagged'

# tourism = gpd.read_feather(os.path.join(CFG.data_path,file_name + '.feather'))


In [44]:
# tourism.set_crs(crs=4326)

In [45]:
# tourism.crs

correct osmid = -2147483648

In [46]:
weird_osmids = communes[communes.osmid == '-2147483648'].index
display(communes.loc[weird_osmids])
for n, idx in enumerate (weird_osmids):
    communes.loc[idx,'osmid'] = str(-int(communes.loc[idx, 'osmid'] )- (n + 100))
display(communes.loc[weird_osmids])

,codgeo,dep,reg,xcl2154,ycl2154,libgeo,geometry,osmid,x,y
1646,04172,04,93,951739,6307958,Roumoules,"POLYGON ((6.14226 43.78449, 6.15661 43.78732, ...",-2147483648,43.8258394,6.1298863
5809,17276,17,75,441803,6473984,Le Pin,"POLYGON ((-0.29059 45.32664, -0.29069 45.32580...",-2147483648,45.7058217,-0.4474511
6751,21178,21,27,842385,6684735,Valforêt,"POLYGON ((4.91591 47.25085, 4.91034 47.25279, ...",-2147483648,47.2447849,4.8809293
9781,27554,27,28,586107,6891010,La Chapelle-Longueville,"POLYGON ((1.38065 49.08067, 1.38858 49.08620, ...",-2147483648,49.1093602,1.4269352
11936,32100,32,76,471924,6313809,Cazeneuve,"POLYGON ((0.15572 43.87439, 0.15694 43.87078, ...",-2147483648,43.9793542,0.4739444
17006,46292,46,76,608523,6400630,Saint-Simon,"POLYGON ((1.82035 44.68227, 1.82045 44.68362, ...",-2147483648,44.6973462,1.8552987
32437,84015,84,93,873172,6345183,Beaumont-du-Ventoux,"POLYGON ((5.24267 44.20141, 5.23974 44.19780, ...",-2147483648,44.1842023,5.1664171


,codgeo,dep,reg,xcl2154,ycl2154,libgeo,geometry,osmid,x,y
1646,04172,04,93,951739,6307958,Roumoules,"POLYGON ((6.14226 43.78449, 6.15661 43.78732, ...",2147483548,43.8258394,6.1298863
5809,17276,17,75,441803,6473984,Le Pin,"POLYGON ((-0.29059 45.32664, -0.29069 45.32580...",2147483547,45.7058217,-0.4474511
6751,21178,21,27,842385,6684735,Valforêt,"POLYGON ((4.91591 47.25085, 4.91034 47.25279, ...",2147483546,47.2447849,4.8809293
9781,27554,27,28,586107,6891010,La Chapelle-Longueville,"POLYGON ((1.38065 49.08067, 1.38858 49.08620, ...",2147483545,49.1093602,1.4269352
11936,32100,32,76,471924,6313809,Cazeneuve,"POLYGON ((0.15572 43.87439, 0.15694 43.87078, ...",2147483544,43.9793542,0.4739444
17006,46292,46,76,608523,6400630,Saint-Simon,"POLYGON ((1.82035 44.68227, 1.82045 44.68362, ...",2147483543,44.6973462,1.8552987
32437,84015,84,93,873172,6345183,Beaumont-du-Ventoux,"POLYGON ((5.24267 44.20141, 5.23974 44.19780, ...",2147483542,44.1842023,5.1664171


In [47]:
communes = communes[['osmid','x','y','codgeo','dep','reg','geometry']]
communes.head(5)

,osmid,x,y,codgeo,dep,reg,geometry
0,142240,46.1502427,4.9202187,01001,01,84,"POLYGON ((4.90497 46.16079, 4.90280 46.15795, ..."
1,147124,46.0070996,5.4233574,01002,01,84,"POLYGON ((5.40440 46.00778, 5.41628 46.00056, ..."
2,145704,45.9584309,5.3595099,01004,01,84,"POLYGON ((5.38637 45.93121, 5.38892 45.93320, ..."
3,148821,45.9986509,4.9026418,01005,01,84,"POLYGON ((4.88657 46.01602, 4.88808 46.01061, ..."
4,146573,45.7494,5.60137,01006,01,84,"POLYGON ((5.59066 45.75997, 5.58396 45.76274, ..."


In [48]:
G = ox.load_graphml(os.path.join(CFG.data_path, f'[{CFG.area_name}]communes_temp_graph_with_only_nodes.graphml'))


In [49]:
# communes.osmid = communes.osmid.astype(str)
communes = communes.set_index('osmid')


In [50]:
geom = communes.loc['142240','geometry']

In [51]:
communes[communes.touches(geom)]

,x,y,codgeo,dep,reg,geometry
osmid,,,,,,
142161,46.1104066,4.9014438,01028,01,84,"POLYGON ((4.90921 46.12923, 4.90261 46.12659, ..."
142241,46.1191397,4.9580413,01093,01,84,"POLYGON ((4.92620 46.11996, 4.92837 46.11300, ..."
142237,46.1402717,4.8997271,01146,01,84,"POLYGON ((4.90280 46.15795, 4.89355 46.14907, ..."
140330,46.1881793,4.8883585,01188,01,84,"POLYGON ((4.93290 46.18339, 4.93365 46.18410, ..."
142236,46.1474,4.86619,01351,01,84,"POLYGON ((4.90497 46.16079, 4.90298 46.16162, ..."
140762,46.1766447,4.9640637,01412,01,84,"POLYGON ((4.95837 46.15325, 4.96394 46.15596, ..."


In [52]:
_, nearest = communes.sindex.nearest(geom)
communes.iloc[nearest]

,x,y,codgeo,dep,reg,geometry
osmid,,,,,,
142161,46.1104066,4.9014438,01028,01,84,"POLYGON ((4.90921 46.12923, 4.90261 46.12659, ..."
142241,46.1191397,4.9580413,01093,01,84,"POLYGON ((4.92620 46.11996, 4.92837 46.11300, ..."
142237,46.1402717,4.8997271,01146,01,84,"POLYGON ((4.90280 46.15795, 4.89355 46.14907, ..."
142236,46.1474,4.86619,01351,01,84,"POLYGON ((4.90497 46.16079, 4.90298 46.16162, ..."
142240,46.1502427,4.9202187,01001,01,84,"POLYGON ((4.90497 46.16079, 4.90280 46.15795, ..."
140762,46.1766447,4.9640637,01412,01,84,"POLYGON ((4.95837 46.15325, 4.96394 46.15596, ..."
140330,46.1881793,4.8883585,01188,01,84,"POLYGON ((4.93290 46.18339, 4.93365 46.18410, ..."


In [53]:
# longest = []
# for idx in communes.index:
#     if len(idx)>len(longest):
#         longest = idx
# longest        
# communes.loc[longest,'codgeo']
# for node in tqdm(G.nodes):
#     if node <0:
#         print(node)

In [54]:
isolated_nodes = []
for node in tqdm(G.nodes):
    geom = communes.loc[str(node),'geometry']
    adjacentes = communes[communes.touches(geom)].index
    # print(adjacentes)
    if len(adjacentes) == 0:
        # print(node,adjacentes)
        # _, nearest = communes.sindex.nearest(geom)
        # adjacentes = [communes.index[nearest[0]]]
        # print(node,adjacentes)
        isolated_nodes.append(node)
    for v in adjacentes:
        G.add_edge(
                    node,
                    int(v),
                    0,
        )
    # break
isolated_nodes    

  0%|          | 0/34466 [00:00<?, ?it/s]

[156066,
 659733,
 960821,
 192873,
 113454,
 107972,
 417923,
 399669,
 215401,
 900558,
 215400,
 278723]

In [55]:
Edges = ox.graph_to_gdfs(G, nodes=False, edges=True)
Edges.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
MultiIndex: 205450 entries, (2531584, 2176035, 0) to (2741108, 2740963, 0)
Data columns (total 1 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   geometry  205450 non-null  geometry
dtypes: geometry(1)
memory usage: 5.9 MB


In [56]:
joblib.dump(Edges,os.path.join(CFG.data_path, f'[{CFG.area_name}]Gedges_communes.pkl'))


['./osm_datas/[france]Gedges_communes.pkl']

In [57]:
import folium

m = folium.Map(location= [48.8, 2.3],
                    width='100%', height='100%', left='00%', top='0%', 
                    position='relative', 
                    tiles= 'OpenStreetMap',     #OpenStreetMap', Stamen
                    attr=None,    #  None   Terrain, Toner, and Watercolor
                    min_zoom=4, max_zoom=20, zoom_start=6, 
                    min_lat= 41, max_lat=52, min_lon=- 4, max_lon=10, 
                    max_bounds=True, 
                    crs='EPSG3857', 
                    control_scale=True, prefer_canvas=False, 
                    no_touch=False, disable_3d=True, 
                    png_enabled=False, zoom_control=True)


for id in isolated_nodes:
    folium.Marker([communes.loc[str(id),'x'],communes.loc[str(id),'y']]).add_to(m)

display(m)
m.save(os.path.join(CFG.folium_path, f'[{CFG.area_name}]tourism_isolated_nodes_check.html'))
del(m)

------------------------------------